In [ ]:
import pandas as pd
import computer_cap
from scipy.stats import pearsonr, spearmanr

In [13]:
df = pd.read_csv('laptop_data.csv')
df.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


Code below takes out the columns i didn't need and changes the Ram column to delete the GB behing the numbers

In [5]:
copy_df = df.copy()
copy_df = copy_df.drop(columns=['Unnamed: 0', 'TypeName', 'ScreenResolution', 'OpSys', 'Weight'])
copy_df['Ram'] = copy_df['Ram'].str.replace('GB', '')
copy_df['Ram'] = pd.to_numeric(copy_df['Ram'])
copy_df.head()

,Company,Inches,Cpu,Ram,Memory,Gpu,Price
0,Apple,13.3,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,71378.6832
1,Apple,13.3,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,47895.5232
2,HP,15.6,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,30636.0000
3,Apple,15.4,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,135195.3360
4,Apple,13.3,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,96095.8080


Here i took the copy database and made another database that only has the screen size column in it. Then i deleted the all the screen sizes that had low amounts of data and only used the top 5 sizes by quanity. 

In [6]:
screen_copy_df = copy_df.copy()
screen_copy_df = screen_copy_df.drop(columns=['Company', 'Cpu', 'Ram', 'Memory', 'Gpu', 'Price'])
sizes_to_delete = [12, 13.5, 13.9, 12.3, 10.1, 15.4, 15.0, 13, 18.4, 17, 14.1, 11.3]
screen_copy_df = screen_copy_df[~screen_copy_df['Inches'].isin(sizes_to_delete)]
screen_copy_df.head()

,Inches
0,13.3
1,13.3
2,15.6
4,13.3
5,15.6


In the database the price was in Indian Rubees and I had to change it to American currency. 

In [7]:
exchange_rate_inr_to_usd = 0.012

def convert_to_usd(Price):
    return Price * exchange_rate_inr_to_usd

copy_df['Price in Dollars'] = copy_df['Price'].apply(convert_to_usd)

copy_df.head()

,Company,Inches,Cpu,Ram,Memory,Gpu,Price,Price in Dollars
0,Apple,13.3,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,71378.6832,856.544198
1,Apple,13.3,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,47895.5232,574.746278
2,HP,15.6,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,30636.0000,367.632000
3,Apple,15.4,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,135195.3360,1622.344032
4,Apple,13.3,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,96095.8080,1153.149696


In [ ]:
copy_df.to_csv('copy_df.csv',  index=False)
screen_copy_df.to_csv('screen_copy_df', index=False)

In [12]:
razer_laptops = copy_df[copy_df['Company'] == 'Razer']
total_razer_laptops = len(razer_laptops)
razer_nvidia_laptops = razer_laptops[razer_laptops['Gpu'].str.startswith('Nvidia')]
razer_nvidia_count = len(razer_nvidia_laptops)
percentage_razer_nvidia = (razer_nvidia_count / total_razer_laptops) * 100
print(f"The percentage of 'Razer' laptops with 'Nvidia' GPUs is: {percentage_razer_nvidia:.2f}%")
spearman_corr, p_value = spearmanr(razer_nvidia_laptops['Price'], range(len(razer_nvidia_laptops)))

print(f"Spearman correlation coefficient: {spearman_corr}")
print(f"P-value: {p_value}")

The percentage of 'Razer' laptops with 'Nvidia' GPUs is: 71.43%
Spearman correlation coefficient: -0.09999999999999999
P-value: 0.8728885715695383


In [ ]:
correlation_coefficient, p_value = spearmanr(copy_df['Price in Dollars'], copy_df['Ram'])
print(f"Spearman correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")

Spearman correlation coefficient: 0.7692698147924433
P-value: 2.2277822110349674e-255
